In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
class KMeans:
    def __init__(self,K,max_iter = 100,diff = 1e-4):
        self.k = K
        self.max_iter = max_iter
        self.diff = diff
        self.centroids = None

    def centroids_init(self,X):
        self.centroids = X[np.random.choice(X.shape[0],self.k,replace=False)]
    
    def compute_distances(self, X, centroids):
        return np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)

    def assign_cluster(self,X):
        distances = self.compute_distances(X,self.centroids)
        cluster = np.argmin(distances,axis=1)
        return cluster
    
    def update_centroids(self,X,labels):
        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(self.k)])
        return new_centroids

    def compute_objective(self, X, labels):
        distances = np.linalg.norm(X - self.centroids[labels], axis=1)
        return np.sum(distances**2)
    
    def fit(self, X):
        self.centroids_init(X)
        for i in range(self.max_iter):
            cluster_assignments = self.assign_cluster(X)
            new_centroids = self.update_centroids(X, cluster_assignments)
            if np.all(np.abs(new_centroids - self.centroids) < self.diff):
                print(f"Converged after {i+1} iterations.")
                break
            
            self.centroids = new_centroids
        return cluster_assignments, self.centroids


In [15]:
def evaluate_objective(X, y, K_values):
    for K in K_values:
        print(f"\nEvaluating for K = {K}")
        kmeans = KMeans(K=K)
        cluster_assignments, centroids = kmeans.fit(X)
        objective = kmeans.compute_objective(X, cluster_assignments)
        print(f"KMeans Objective Function Value: {objective:.4f}")
        

def compute_purity(y_true, cluster_assignments, K):
    cluster_to_label = {}
    for cluster in range(K):
        indices = np.where(cluster_assignments == cluster)[0]
        cluster_labels = y_true[indices]
        if len(cluster_labels) > 0:
            most_common_label = Counter(cluster_labels).most_common(1)[0][0]
            cluster_to_label[cluster] = most_common_label
    
    y_pred = np.array([cluster_to_label[c] for c in cluster_assignments])
    accuracy = np.mean(y_pred == y_true)
    return accuracy

def compute_gini_index(y_true, cluster_assignments, K):
    total_samples = len(y_true)
    gini_total = 0

    for cluster in range(K):
        indices = np.where(cluster_assignments == cluster)[0]
        cluster_labels = y_true[indices]
        if len(cluster_labels) == 0:
            continue
        
        label_counts = Counter(cluster_labels)
        cluster_size = len(cluster_labels)
        gini = 1 - sum((count / cluster_size) ** 2 for count in label_counts.values())
        gini_total += (cluster_size / total_samples) * gini
    
    return gini_total


In [10]:
categories = None
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

X_texts = newsgroups.data 
y_labels = newsgroups.target


tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X_texts)

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y_labels, test_size=0.2, random_state=42, stratify=y_labels
)

X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

print(f"Training Data Shape: {X_train.shape}, Training Labels Shape: {y_train.shape}")
print(f"Test Data Shape: {X_test.shape}, Test Labels Shape: {y_test.shape}")


Training Data Shape: (15076, 5000), Training Labels Shape: (15076,)
Test Data Shape: (3770, 5000), Test Labels Shape: (3770,)


In [11]:
K_values = [10, 5, 20]
evaluate_objective(X_train_dense, y_train, K_values)


Evaluating for K = 10
Converged after 34 iterations.
KMeans Objective Function Value: 14151.3803

Evaluating for K = 5
Converged after 39 iterations.
KMeans Objective Function Value: 14262.3983

Evaluating for K = 20


/tmp/ipykernel_45654/3664018627.py:21: RuntimeWarning: Mean of empty slice.
  new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(self.k)])
/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


KMeans Objective Function Value: 14414.4079


In [16]:

for K in K_values:
    print(f"\nEvaluating Purity and Gini Index for K = {K}")
    kmeans = KMeans(K=K)
    cluster_assignments, centroids = kmeans.fit(X_train_dense)
    purity = compute_purity(y_train, cluster_assignments, K)
    gini_index = compute_gini_index(y_train, cluster_assignments, K)

    print(f"Purity Score: {purity:.4f}")
    print(f"Gini Index: {gini_index:.4f}")


Evaluating Purity and Gini Index for K = 10
Converged after 18 iterations.
Purity Score: 0.0706
Gini Index: 0.9369

Evaluating Purity and Gini Index for K = 5
Converged after 32 iterations.
Purity Score: 0.1596
Gini Index: 0.8909

Evaluating Purity and Gini Index for K = 20
Converged after 38 iterations.
Purity Score: 0.0990
Gini Index: 0.9196


In [ ]:
from sklearn.cluster import KMeans
for K in K_values:
    print(f"\nEvaluating Purity and Gini Index for K = {K}")

    kmeans = KMeans(n_clusters=K, random_state=42, n_init=10, max_iter=300)
    cluster_assignments = kmeans.fit_predict(X_train)

    purity = compute_purity(y_train, cluster_assignments, K)
    gini_index = compute_gini_index(y_train, cluster_assignments, K)

    print(f"Purity Score: {purity:.4f}")
    print(f"Gini Index: {gini_index:.4f}")


Evaluating Purity and Gini Index for K = 10
Purity Score: 0.2544
Gini Index: 0.8216

Evaluating Purity and Gini Index for K = 5
Purity Score: 0.1764
Gini Index: 0.8762

Evaluating Purity and Gini Index for K = 20
Purity Score: 0.3358
Gini Index: 0.7540
